In [1]:
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
from pandas import DataFrame
import pandas_profiling
from time import sleep
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import requests
import datetime
import requests
import glob   
import os
import io
import re
from sklearn import svm
from process_craigslist import Process
from scrape_craigslist import Scrape

In [2]:
import pickle
from scipy import stats
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression

In [3]:
li=[]
for filename in glob.glob(r"C:\Users\Silmi\Desktop\UsedCar-PriceEstimator\data\*.csv"):
    collection = pd.read_csv(filename, index_col=None, header=0)
    li.append(collection)

frame = pd.concat(li, axis=0, ignore_index=True)

In [4]:
#Process(bigframe dataframe,makmod str)
#_df=Process(frame,'hyundai elantra')
_df=Process(frame)
#_df['Make&Model'].value_counts()[56:66]
_df.shape

(21912, 6)

In [5]:
_df=Process(frame,'toyota corolla')

In [6]:
profile=_df.profile_report()

In [7]:
#profile

In [8]:
X=_df.drop(columns='price:')
y=_df['price:']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=47)
scaler = StandardScaler()
scaler.fit(X_train)
X_tr_scaled = scaler.transform(X_train)
X_te_scaled = scaler.transform(X_test)
Rm = RandomForestRegressor().fit(X_tr_scaled, y_train)
#clf = svm.SVC()
#clf.fit(X_tr_scaled, y_train)
y_tr_pred = Rm.predict(X_tr_scaled)
y_te_pred = Rm.predict(X_te_scaled)

In [9]:
median_r2 = r2_score(y_train, y_tr_pred)

In [10]:
median_r2

0.8540326875370348

In [16]:
final_lin=['https://sfbay.craigslist.org/eby/cto/d/martinez-2018-toyota-corolla-le-sedan/7263747364.html']

In [17]:
test=Scrape(final_lin)
feed=Process(test,'toyota corolla')

In [18]:
feed

,year:,condition:,odometer:,status:,price:
0,4,3,45000,2,13900


In [19]:

x_actual=feed.drop(columns='price:')
x_values = scaler.transform(x_actual)
y_actual=feed['price:']
pred_price = Rm.predict(x_values)
pred_price[0]
rate=(y_actual)/(pred_price[0])
rate[0]

1.0048308992489552

In [20]:
print(f"actul:{y_actual}",f"predicted: {int(pred_price[0])}")

actul:0    13900
Name: price:, dtype: int32 predicted: 13833
